In [39]:
from collections import defaultdict
from itertools import product
import json

import pandas as pd
import torch

import roach
from relbench.base import TaskType
from relbench.datasets import get_dataset_names
from relbench.tasks import get_task_names, get_task

In [2]:
all_stores = roach.scan("relbench/2024-07-05")

In [3]:
len(all_stores)

549

In [4]:
all_stores[-1]["__roach__"]

{'project': 'relbench/2024-07-05',
 'timestamp': 1720376613254945500,
 'caller_file': 'idgnn_link.py',
 'done': True}

In [5]:
def wrap(name):
    return r"\texttt{" + name + r"}"

In [6]:
txt = {
    "val": "Val",
    "test": "Test",
}

# classification

In [26]:
metric = "roc_auc"
higher_is_better = True

In [27]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value not in [
            TaskType.BINARY_CLASSIFICATION.value,
            TaskType.MULTICLASS_CLASSIFICATION.value,
        ]:
            continue
        for script in [
            "gnn_node",
            "lightgbm_node",
            "hybrid_node",
        ]:
            for split in [
                "val",
                "test",
            ]:
                vals = []
                for seed in range(5):
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task
                            and store["args"]["seed"] == seed,
                            all_stores,
                        )
                    )
                    try:
                        store = stores[-1]
                        val = store[split][metric]
                        vals.append(val)
                    except IndexError:
                        # val = float("nan")
                        # vals.append(val)
                        pass
                val = torch.tensor(vals)
                mean = val.mean().item()
                std = val.std().item()
                record = {
                    "dataset": dataset,
                    "task": task,
                    "script": script,
                    "split": split,
                    "mean": mean,
                    "std": std,
                }
                table_data.append(record)

In [28]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    if script == "hybrid_node":
        continue

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    tex_val = (
        r"$"
        + opt_bm_open
        + f"{mean * 100:.2f}"
        + opt_bm_close
        + r"_{\pm "
        + f"{std * 100:.2f}"
        + r"}$"
    )

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-churn}      Val   $\bm{70.06}_{\pm 0.04}$   
                                             Test  $\bm{70.20}_{\pm 0.06}$   
                    \texttt{item-churn}      Val   $\bm{81.04}_{\pm 0.05}$   
                                             Test  $\bm{81.46}_{\pm 0.07}$   
\texttt{rel-avito}  \texttt{user-visits}     Val   $\bm{69.64}_{\pm 0.04}$   
                                             Test  $\bm{66.19}_{\pm 0.11}$   
                    \texttt{user-clicks}     Val   $\bm{64.64}_{\pm 0.30}$   
                                             Test  $\bm{65.60}_{\pm 2.11}$   
\texttt{rel-event}  \texttt{user-repeat}     Val   $\bm{72.02}_{\pm 2.44}$   
                                             Test  $\bm{78.80}_{\pm 0.72}$   
                    \texttt{user-ignore}     Val        $90.82_{\pm 0.74}$   
                                             Test  $\bm{80.11}_{\pm 1.59}$   
\texttt{rel-f1}     \texttt{driver-dnf}      Val   $\bm{71.69}_{\pm 1.56}$   
                                             Test  $\bm{72.51}_{\pm 0.14}$   
                    \texttt{driver-top3}     Val   $\bm{77.26}_{\pm 3.52}$   
                                             Test  $\bm{75.47}_{\pm 0.70}$   
\texttt{rel-hm}     \texttt{user-churn}      Val   $\bm{70.44}_{\pm 0.09}$   
                                             Test  $\bm{69.97}_{\pm 0.10}$   
\texttt{rel-stack}  \texttt{user-engagement} Val   $\bm{90.21}_{\pm 0.08}$   
                                             Test  $\bm{90.58}_{\pm 0.10}$   
                    \texttt{user-badge}      Val   $\bm{89.88}_{\pm 0.09}$   
                                             Test  $\bm{88.88}_{\pm 0.08}$   
\texttt{rel-trial}  \texttt{study-outcome}   Val   $\bm{68.38}_{\pm 0.23}$   
                                             Test  $\bm{68.76}_{\pm 1.09}$   

                                                             lightgbm_node  
\texttt{rel-amazon} \texttt{user-churn}      Val        $52.00_{\pm 0.02}$  
                                             Test       $52.34_{\pm 0.07}$  
                    \texttt{item-churn}      Val        $61.27_{\pm 0.20}$  
                                             Test       $61.96_{\pm 0.26}$  
\texttt{rel-avito}  \texttt{user-visits}     Val        $53.31_{\pm 0.09}$  
                                             Test       $53.05_{\pm 0.32}$  
                    \texttt{user-clicks}     Val        $55.63_{\pm 0.31}$  
                                             Test       $53.60_{\pm 0.59}$  
\texttt{rel-event}  \texttt{user-repeat}     Val        $67.76_{\pm 1.10}$  
                                             Test       $69.74_{\pm 2.17}$  
                    \texttt{user-ignore}     Val        $86.62_{\pm 0.38}$  
                                             Test  $\bm{78.19}_{\pm 0.43}$  
\texttt{rel-f1}     \texttt{driver-dnf}      Val        $68.42_{\pm 1.14}$  
                                             Test  $\bm{68.56}_{\pm 3.89}$  
                    \texttt{driver-top3}     Val        $67.76_{\pm 2.75}$  
                                             Test  $\bm{73.92}_{\pm 5.75}$  
\texttt{rel-hm}     \texttt{user-churn}      Val        $56.05_{\pm 0.05}$  
                                             Test       $55.21_{\pm 0.12}$  
\texttt{rel-stack}  \texttt{user-engagement} Val        $65.12_{\pm 0.25}$  
                                             Test       $63.39_{\pm 0.26}$  
                    \texttt{user-badge}      Val        $65.39_{\pm 0.05}$  
                                             Test       $63.43_{\pm 0.12}$  
\texttt{rel-trial}  \texttt{study-outcome}   Val   $\bm{68.30}_{\pm 0.53}$  
                                             Test  $\bm{70.09}_{\pm 1.41}$

In [29]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-5} \cmidrule{2-5}", r"\cmidrule{1-5}")
print(tex)

\begin{tabular}{lllll}
\toprule
 &  &  & gnn_node & lightgbm_node \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-churn}} & Val & $\bm{70.06}_{\pm 0.04}$ & $52.00_{\pm 0.02}$ \\
 &  & Test & $\bm{70.20}_{\pm 0.06}$ & $52.34_{\pm 0.07}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{item-churn}} & Val & $\bm{81.04}_{\pm 0.05}$ & $61.27_{\pm 0.20}$ \\
 &  & Test & $\bm{81.46}_{\pm 0.07}$ & $61.96_{\pm 0.26}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{user-visits}} & Val & $\bm{69.64}_{\pm 0.04}$ & $53.31_{\pm 0.09}$ \\
 &  & Test & $\bm{66.19}_{\pm 0.11}$ & $53.05_{\pm 0.32}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{user-clicks}} & Val & $\bm{64.64}_{\pm 0.30}$ & $55.63_{\pm 0.31}$ \\
 &  & Test & $\bm{65.60}_{\pm 2.11}$ & $53.60_{\pm 0.59}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-event}} & \multirow[c]{2}{*}{\texttt{user-repeat}} & Val & $\bm{72.02}_{\pm 2.44}$ & $67.76_{\pm 1.10}$ \\
 &  &

In [40]:
lb_sub = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    if split != "test":
        continue
    
    lb_sub[script][f"{dataset}/{task}"] = [mean, std]
print(json.dumps(lb_sub, indent=2))

{
  "gnn_node": {
    "rel-amazon/user-ltv": [
      7.959512105677888,
      0.018910855950465336
    ],
    "rel-amazon/item-ltv": [
      76.71226675772242,
      0.6715122676553593
    ],
    "rel-avito/ad-ctr": [
      0.04096491022415112,
      0.0006861592886295446
    ],
    "rel-event/user-attendance": [
      0.2616523967571157,
      0.0019310083017786213
    ],
    "rel-f1/driver-position": [
      3.9789279173340715,
      0.08295935325229917
    ],
    "rel-hm/item-sales": [
      0.05552658133966328,
      0.0002496324294060044
    ],
    "rel-stack/post-votes": [
      0.06512722107050253,
      0.00014255270181550073
    ],
    "rel-trial/study-adverse": [
      44.46586891707132,
      0.2403561027047603
    ],
    "rel-trial/site-success": [
      0.3950502638864144,
      0.01858076961697409
    ]
  },
  "lightgbm_node": {
    "rel-amazon/user-ltv": [
      8.85977882288307,
      0.0
    ],
    "rel-amazon/item-ltv": [
      92.96326307019072,
      0.5302020577311

# regression

In [33]:
metric = "mae"
higher_is_better = False

In [34]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.REGRESSION.value:
            continue
        for script in [
            "gnn_node",
            "lightgbm_node",
            "hybrid_node",
            "baseline_node",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "baseline_node":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_zero",
                        "global_mean",
                        "global_median",
                        "entity_mean",
                        "entity_median",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

In [35]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    if script == "hybrid_node":
        continue

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_node",
        "lightgbm_node",
        # "hybrid_node",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean:.3f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std:.3f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean:.3f}" + opt_bm_close + r"$"

    rec["tex"] = tex_val

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val

tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-ltv}        Val    $\bm{7.039}_{\pm 0.007}$   
                                             Test   $\bm{7.960}_{\pm 0.019}$   
                    \texttt{item-ltv}        Val        $68.012_{\pm 0.255}$   
                                             Test  $\bm{76.712}_{\pm 0.672}$   
\texttt{rel-avito}  \texttt{ad-ctr}          Val         $0.037_{\pm 0.000}$   
                                             Test   $\bm{0.041}_{\pm 0.001}$   
\texttt{rel-event}  \texttt{user-attendance} Val         $0.258_{\pm 0.003}$   
                                             Test   $\bm{0.262}_{\pm 0.002}$   
\texttt{rel-f1}     \texttt{driver-position} Val    $\bm{3.188}_{\pm 0.024}$   
                                             Test   $\bm{3.979}_{\pm 0.083}$   
\texttt{rel-hm}     \texttt{item-sales}      Val         $0.065_{\pm 0.000}$   
                                             Test   $\bm{0.056}_{\pm 0.000}$   
\texttt{rel-stack}  \texttt{post-votes}      Val         $0.059_{\pm 0.000}$   
                                             Test   $\bm{0.065}_{\pm 0.000}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val        $46.216_{\pm 0.295}$   
                                             Test  $\bm{44.466}_{\pm 0.240}$   
                    \texttt{site-success}    Val    $\bm{0.402}_{\pm 0.010}$   
                                             Test   $\bm{0.395}_{\pm 0.019}$   

                                                               lightgbm_node  \
\texttt{rel-amazon} \texttt{user-ltv}        Val         $7.730_{\pm 0.000}$   
                                             Test        $8.860_{\pm 0.000}$   
                    \texttt{item-ltv}        Val        $83.322_{\pm 0.382}$   
                                             Test       $92.963_{\pm 0.530}$   
\texttt{rel-avito}  \texttt{ad-ctr}          Val         $0.037_{\pm 0.000}$   
                                             Test   $\bm{0.041}_{\pm 0.000}$   
\texttt{rel-event}  \texttt{user-attendance} Val         $0.262_{\pm 0.000}$   
                                             Test   $\bm{0.264}_{\pm 0.000}$   
\texttt{rel-f1}     \texttt{driver-position} Val         $3.450_{\pm 0.030}$   
                                             Test   $\bm{4.170}_{\pm 0.137}$   
\texttt{rel-hm}     \texttt{item-sales}      Val         $0.086_{\pm 0.000}$   
                                             Test        $0.076_{\pm 0.000}$   
\texttt{rel-stack}  \texttt{post-votes}      Val         $0.062_{\pm 0.000}$   
                                             Test        $0.068_{\pm 0.000}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val        $45.774_{\pm 1.191}$   
                                             Test  $\bm{44.011}_{\pm 0.998}$   
                    \texttt{site-success}    Val         $0.417_{\pm 0.003}$   
                                             Test        $0.425_{\pm 0.003}$   

                                                    global_zero global_mean  \
\texttt{rel-amazon} \texttt{user-ltv}        Val        $7.730$    $13.559$   
                                             Test       $8.860$    $14.080$   
                    \texttt{item-ltv}        Val      $104.830$   $106.235$   
                                             Test     $115.584$   $112.906$   
\texttt{rel-avito}  \texttt{ad-ctr}          Val        $0.048$     $0.048$   
                                             Test       $0.052$     $0.051$   
\texttt{rel-event}  \texttt{user-attendance} Val        $0.262$     $0.457$   
                                             Test  $\bm{0.264}$     $0.470$   
\texttt{rel-f1}     \texttt{driver-position} Val       $11.083$     $4.334$   
                                             Test      $11.926$     $4.513$   
\texttt{rel-hm}     \texttt{item-sales}      Val        $0.086$     $0.142$   
                                             Test       $0.076$     $0.134$   

In [36]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-10} \cmidrule{2-10}", r"\cmidrule{1-10}")
print(tex)

\begin{tabular}{llllllllll}
\toprule
 &  &  & gnn_node & lightgbm_node & global_zero & global_mean & global_median & entity_mean & entity_median \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-ltv}} & Val & $\bm{7.039}_{\pm 0.007}$ & $7.730_{\pm 0.000}$ & $7.730$ & $13.559$ & $7.730$ & $11.595$ & $10.218$ \\
 &  & Test & $\bm{7.960}_{\pm 0.019}$ & $8.860_{\pm 0.000}$ & $8.860$ & $14.080$ & $8.860$ & $12.075$ & $10.725$ \\
\cmidrule{2-10}
 & \multirow[c]{2}{*}{\texttt{item-ltv}} & Val & $68.012_{\pm 0.255}$ & $83.322_{\pm 0.382}$ & $104.830$ & $106.235$ & $89.349$ & $109.973$ & $96.793$ \\
 &  & Test & $\bm{76.712}_{\pm 0.672}$ & $92.963_{\pm 0.530}$ & $115.584$ & $112.906$ & $99.310$ & $109.965$ & $95.367$ \\
\cmidrule{1-10}
\multirow[c]{2}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{ad-ctr}} & Val & $0.037_{\pm 0.000}$ & $0.037_{\pm 0.000}$ & $0.048$ & $0.048$ & $0.040$ & $0.044$ & $0.044$ \\
 &  & Test & $\bm{0.041}_{\pm 0.001}$ & $\bm{0.

In [40]:
lb_sub = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    if split != "test":
        continue
    
    lb_sub[script][f"{dataset}/{task}"] = [mean, std]
print(json.dumps(lb_sub, indent=2))

{
  "gnn_node": {
    "rel-amazon/user-ltv": [
      7.959512105677888,
      0.018910855950465336
    ],
    "rel-amazon/item-ltv": [
      76.71226675772242,
      0.6715122676553593
    ],
    "rel-avito/ad-ctr": [
      0.04096491022415112,
      0.0006861592886295446
    ],
    "rel-event/user-attendance": [
      0.2616523967571157,
      0.0019310083017786213
    ],
    "rel-f1/driver-position": [
      3.9789279173340715,
      0.08295935325229917
    ],
    "rel-hm/item-sales": [
      0.05552658133966328,
      0.0002496324294060044
    ],
    "rel-stack/post-votes": [
      0.06512722107050253,
      0.00014255270181550073
    ],
    "rel-trial/study-adverse": [
      44.46586891707132,
      0.2403561027047603
    ],
    "rel-trial/site-success": [
      0.3950502638864144,
      0.01858076961697409
    ]
  },
  "lightgbm_node": {
    "rel-amazon/user-ltv": [
      8.85977882288307,
      0.0
    ],
    "rel-amazon/item-ltv": [
      92.96326307019072,
      0.5302020577311

# link prediction

In [13]:
metric = "link_prediction_map"
higher_is_better = True

In [15]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.LINK_PREDICTION.value:
            continue
        for script in [
            "gnn_link",
            "idgnn_link",
            "lightgbm_link",
            "baseline_link",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "baseline_link":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_popularity",
                        "past_visit",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

In [18]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_link",
        "idgnn_link",
        "lightgbm_link",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean * 100:.2f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std * 100:.2f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean * 100:.2f}" + opt_bm_close + r"$"

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val   $\bm{2.50}_{\pm 0.16}$   
                                                   Test  $\bm{1.51}_{\pm 0.13}$   
                    \texttt{user-item-rate}        Val   $\bm{2.41}_{\pm 0.15}$   
                                                   Test  $\bm{1.58}_{\pm 0.19}$   
                    \texttt{user-item-review}      Val   $\bm{1.94}_{\pm 0.10}$   
                                                   Test  $\bm{0.98}_{\pm 0.08}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val        $0.08_{\pm 0.02}$   
                                                   Test       $0.02_{\pm 0.00}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val        $0.91_{\pm 0.04}$   
                                                   Test       $0.80_{\pm 0.03}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val        $0.43_{\pm 0.09}$   
                                                   Test       $0.12_{\pm 0.05}$   
                    \texttt{post-post-related}     Val        $0.00_{\pm 0.01}$   
                                                   Test       $0.04_{\pm 0.02}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val        $3.05_{\pm 0.20}$   
                                                   Test       $2.87_{\pm 0.44}$   
                    \texttt{site-sponsor-run}      Val       $13.77_{\pm 0.89}$   
                                                   Test      $10.03_{\pm 0.86}$   

                                                                      idgnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val         $0.19_{\pm 0.00}$   
                                                   Test        $0.16_{\pm 0.00}$   
                    \texttt{user-item-rate}        Val         $0.22_{\pm 0.00}$   
                                                   Test        $0.18_{\pm 0.00}$   
                    \texttt{user-item-review}      Val         $0.15_{\pm 0.00}$   
                                                   Test        $0.13_{\pm 0.00}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val    $\bm{5.40}_{\pm 0.02}$   
                                                   Test   $\bm{3.66}_{\pm 0.03}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val    $\bm{2.64}_{\pm 0.00}$   
                                                   Test   $\bm{2.80}_{\pm 0.01}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val   $\bm{15.11}_{\pm 0.06}$   
                                                   Test  $\bm{12.64}_{\pm 0.11}$   
                    \texttt{post-post-related}     Val    $\bm{7.76}_{\pm 0.23}$   
                                                   Test  $\bm{10.89}_{\pm 0.21}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val   $\bm{11.35}_{\pm 0.03}$   
                                                   Test  $\bm{11.34}_{\pm 0.06}$   
                    \texttt{site-sponsor-run}      Val   $\bm{17.41}_{\pm 0.06}$   
                                                   Test  $\bm{19.00}_{\pm 0.14}$   

                                                              lightgbm_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val    $0.47_{\pm 0.12}$   
                                                   Test   $0.35_{\pm 0.08}$   
                    \texttt{user-item-rate}        Val    $0.59_{\pm 0.13}$   
                                                   Test   $0.45_{\pm 0.07}$   
                    \texttt{user-item-review}      Val    $0.40_{\pm 0.14}$   
                                                   Test   $0.25_{\pm 0.06}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val    $0.17_{\pm 0.01}$   
                                                   Test   $0.06_{\pm 0.01}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val    $0.44_{\pm 0.03}$   
                                                   Test   $0.38_{\pm 0.02}$   
\texttt{rel-stack}  \texttt

In [19]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-7} \cmidrule{2-7}", r"\cmidrule{1-7}")
print(tex)

\begin{tabular}{llllllll}
\toprule
 &  &  & gnn_link & idgnn_link & lightgbm_link & global_popularity & past_visit \\
\midrule
\multirow[c]{6}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-item-purchase}} & Val & $\bm{2.50}_{\pm 0.16}$ & $0.19_{\pm 0.00}$ & $0.47_{\pm 0.12}$ & $0.62$ & $0.12$ \\
 &  & Test & $\bm{1.51}_{\pm 0.13}$ & $0.16_{\pm 0.00}$ & $0.35_{\pm 0.08}$ & $0.49$ & $0.10$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-rate}} & Val & $\bm{2.41}_{\pm 0.15}$ & $0.22_{\pm 0.00}$ & $0.59_{\pm 0.13}$ & $0.37$ & $0.14$ \\
 &  & Test & $\bm{1.58}_{\pm 0.19}$ & $0.18_{\pm 0.00}$ & $0.45_{\pm 0.07}$ & $0.48$ & $0.12$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-review}} & Val & $\bm{1.94}_{\pm 0.10}$ & $0.15_{\pm 0.00}$ & $0.40_{\pm 0.14}$ & $0.37$ & $0.07$ \\
 &  & Test & $\bm{0.98}_{\pm 0.08}$ & $0.13_{\pm 0.00}$ & $0.25_{\pm 0.06}$ & $0.25$ & $0.05$ \\
\cmidrule{1-8} \cmidrule{2-8}
\multirow[c]{2}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{

In [40]:
lb_sub = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    if split != "test":
        continue
    
    lb_sub[script][f"{dataset}/{task}"] = [mean, std]
print(json.dumps(lb_sub, indent=2))

{
  "gnn_node": {
    "rel-amazon/user-ltv": [
      7.959512105677888,
      0.018910855950465336
    ],
    "rel-amazon/item-ltv": [
      76.71226675772242,
      0.6715122676553593
    ],
    "rel-avito/ad-ctr": [
      0.04096491022415112,
      0.0006861592886295446
    ],
    "rel-event/user-attendance": [
      0.2616523967571157,
      0.0019310083017786213
    ],
    "rel-f1/driver-position": [
      3.9789279173340715,
      0.08295935325229917
    ],
    "rel-hm/item-sales": [
      0.05552658133966328,
      0.0002496324294060044
    ],
    "rel-stack/post-votes": [
      0.06512722107050253,
      0.00014255270181550073
    ],
    "rel-trial/study-adverse": [
      44.46586891707132,
      0.2403561027047603
    ],
    "rel-trial/site-success": [
      0.3950502638864144,
      0.01858076961697409
    ]
  },
  "lightgbm_node": {
    "rel-amazon/user-ltv": [
      8.85977882288307,
      0.0
    ],
    "rel-amazon/item-ltv": [
      92.96326307019072,
      0.5302020577311